In [4]:
import pandas as pd
import numpy as np

In [14]:
from sklearn.ensemble import ExtraTreesClassifier

In [8]:
from sklearn.model_selection import cross_val_score

In [30]:
path1 = 'train.csv'
path2 = 'test.csv'
submission = 'submission.csv' 

In [59]:
  
def features_modification(data2):
    data = data2
    data['abs_ht'] = data.Elevation-data.Vertical_Distance_To_Hydrology
    data['app_ht'] = data.Elevation+data.Vertical_Distance_To_Hydrology
    data['hydro'] = (data['Horizontal_Distance_To_Hydrology']**2+data['Vertical_Distance_To_Hydrology']**2)**0.5
    data['fire'] = data['Horizontal_Distance_To_Hydrology']+data['Horizontal_Distance_To_Fire_Points']
    data['fire2'] = data['Horizontal_Distance_To_Hydrology']-data['Horizontal_Distance_To_Fire_Points']
    data['road'] = data['Horizontal_Distance_To_Hydrology']+data['Horizontal_Distance_To_Roadways']
    data['road2'] = data['Horizontal_Distance_To_Hydrology']-data['Horizontal_Distance_To_Roadways']
    data['fire_road'] = data['Horizontal_Distance_To_Fire_Points']+data['Horizontal_Distance_To_Roadways']
    data['fire'] = data['Horizontal_Distance_To_Fire_Points']-data['Horizontal_Distance_To_Roadways']
    data['Soil']=0
    for i in range(1,41):
        data['Soil']=data['Soil']+i*data['Soil_Type'+str(i)]
    data['Wilderness_Area']=0
    for i in range(1,5):
        data['Wilderness_Area']=data['Wilderness_Area']+i*data['Wilderness_Area'+str(i)]
      
    return data
  

In [60]:
def get_features():
    return ['Elevation','Aspect','Slope','Horizontal_Distance_To_Hydrology','Vertical_Distance_To_Hydrology',
    'Horizontal_Distance_To_Roadways','Hillshade_9am','Hillshade_Noon','Hillshade_3pm','Horizontal_Distance_To_Fire_Points',
    'abs_ht','app_ht','hydro','fire','fire2','road','road2','fire_road','fire','Soil','Wilderness_Area']


In [180]:
train_df = pd.read_csv(path1)
test_df = pd.read_csv(path2)
train_df = features_modification(train_df)
train_y=train_df.Cover_Type.values   
test_df = features_modification(test_df)
 

In [224]:
features = get_features()
y_train = train_df['Cover_Type'].values
test_id = test_df['Id']
X_train =  train_df[:][features].values 

In [226]:
train_df[:][features]

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,app_ht,hydro,fire,fire2,road,road2,fire_road,fire,Soil,Wilderness_Area
0,2596,51,3,258,0,510,221,232,148,6279,...,2596,258.000000,5769,-6021,768,-252,6789,5769,0,1
1,2590,56,2,212,-6,390,220,235,151,6225,...,2584,212.084889,5835,-6013,602,-178,6615,5835,0,1
2,2804,139,9,268,65,3180,234,238,135,6121,...,2869,275.769832,2941,-5853,3448,-2912,9301,2941,0,1
3,2785,155,18,242,118,3090,238,238,122,6211,...,2903,269.235956,3121,-5969,3332,-2848,9301,3121,0,1
4,2595,45,2,153,-1,391,220,234,150,6172,...,2594,153.003268,5781,-6019,544,-238,6563,5781,0,1
5,2579,132,6,300,-15,67,230,237,140,6031,...,2564,300.374766,5964,-5731,367,233,6098,5964,0,1
6,2606,45,7,270,5,633,222,225,138,6256,...,2611,270.046292,5623,-5986,903,-363,6889,5623,0,1
7,2605,49,4,234,7,573,222,230,144,6228,...,2612,234.104677,5655,-5994,807,-339,6801,5655,0,1
8,2617,45,9,240,56,666,223,221,133,6244,...,2673,246.446749,5578,-6004,906,-426,6910,5578,0,1
9,2612,59,10,247,11,636,228,219,124,6230,...,2623,247.244818,5594,-5983,883,-389,6866,5594,0,1


In [142]:
#test_df=test_df.drop(['Id'],axis=1)

In [198]:
train_y.shape

(15120,)

In [82]:
print(len(train_y))
len(train_df)

15120


15120

In [208]:
test_df[17000:17001][features]

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,app_ht,hydro,fire,fire2,road,road2,fire_road,fire,Soil,Wilderness_Area
17000,2837,63,3,30,0,3710,222,233,146,6342,...,2837,30.0,2632,-6312,3740,-3680,10052,2632,20,1


In [182]:
def split_X_test():
    length = len(test_df)
    n  =  length // 10000
    print(n)
    split_test_data = []
    
    for i in range(n):
    
      split_test_data.append(test_df[i*10000:(i+1)*10000][features].values)
        
    
    split_test_data.append(test_df[n*10000:length][features].values)
      
    return split_test_data
 
X_test = split_X_test()
 

56


In [106]:
from sklearn import ensemble
from sklearn import metrics

In [174]:
'''len(X_test)
X_Test=[]
for i in range(len(X_test)):
    for j in range(len(X_test[i])):
        X_Test.append((X_test[i][j]))'''

In [166]:
a=np.array(X_Test)
a.shape


(565892, 21)

In [176]:
a=np.array(X_train)
a.shape

(15120, 21)

In [177]:
np.array(y_train).shape

(15120,)

In [185]:
clf = ExtraTreesClassifier(max_features=0.3, n_estimators=500)
clf.fit(X_train, y_train)


ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features=0.3, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [214]:
y_predict=[]
for var in X_test:
    y_predict.extend(clf.predict(var))
  

In [223]:
submission = pd.DataFrame(data= {'Id' : test_id, 'Cover_Type': y_predict})
submission.to_csv("submission.csv", index=False)
 

In [191]:
l=np.array(y_test)
l.shape

(3780,)

In [219]:
X_Test=[]
for i in range(len(X_test)):
    for j in range(len(X_test[i])):
        X_Test.append(X_test[i][j])
        

In [ ]:
pd.DataFrame({'Id':test_df.Id.values,'Cover_Type':y_test_predict})\
            .sort_index(ascending=False,axis=1).to_csv('ForestdReport.csv',index=False)

In [ ]:
len(X_test)

In [ ]:
df=pd.read_csv("ForestdReport.csv")

In [ ]:
lol = df.values.tolist()

In [ ]:
len(lol)

In [ ]:
len(y_test)

In [ ]:
len(y_predict)

In [ ]:
len(X_test)

In [ ]:
X_test

In [ ]:
y_test

In [ ]:
X_train